In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense, Flatten
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from sklearn.metrics import confusion_matrix
import tensorflow as tf
import random as rn
import itertools

C:\Users\sjcrum\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(29)  # For numpy numbers
rn.seed(29)   # For Python
tf.set_random_seed(29)    #For Tensorflow

In [3]:
#Force tensorflow to use a single thread
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

In [4]:
os.chdir("C:/Users/sjcrum/Documents/Data_Science_Capstone/MLTutorial")
train_path = "cats_and_dogs/train"
test_path = "cats_and_dogs/test"
valid_path = "cats_and_dogs/valid"

In [5]:
#Generates batches of tensor image data that images must be in
#F-f-d takes path and puts in batches of normalized data, one-hot encodes classes (class_mode argument)
train_batches = ImageDataGenerator().flow_from_directory(train_path, target_size = (224,224), classes = ['dog', 'cat'], batch_size = 10)
test_batches = ImageDataGenerator().flow_from_directory(test_path, target_size = (224,224), classes = ['dog', 'cat'], batch_size = 5)
valid_batches = ImageDataGenerator().flow_from_directory(valid_path, target_size = (224,224), classes = ['dog', 'cat'], batch_size = 5)

Found 350 images belonging to 2 classes.
Found 150 images belonging to 2 classes.
Found 100 images belonging to 2 classes.


In [6]:
#### VGG 16 model ####
vgg16_model = keras.applications.vgg16.VGG16()

#Create a sequential model with the VGG16 keras model functional API
#Iterate over every VGG 16 model and add to sequential type model
model = Sequential()

for layer in vgg16_model.layers[:-1]:  #Removes the last layers with 1000 classes
    model.add(layer)
    
for layer in model.layers:
    layer.trainable = False #Freeze a layer so weights arent updated, use for finetuning just the end
    
model.add(Dense(2, activation = 'softmax'))

vgg16_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [ ]:
#### Training VGG 16 Model ####

model.compile(Adam(lr = 0.0001), loss = 'categorical_crossentropy', metrics = ['accuracy'])

model.fit_generator(train_batches, steps_per_epoch=35, validation_data = valid_batches, 
                    validation_steps=35, epochs = 5, verbose = 2)

Epoch 1/5


In [ ]:
predictions = model.predict_generator(test_batches, steps = 15, verbose = 0)
predictions